# Data pre-processing for dublin-bus datasets

In [53]:
import os
import sys
import pandas as pd
import progressbar
import json

## Read data into a dataframe

In [19]:
header = ['Timestamp','Line_ID','Direction','Journey_Pattern_ID','Time_Frame','Vehicle_Journey_ID', \
         'Operator','Congestion','Lon','Lat','Delay','Block_ID','Vehicle_ID','Stop_ID','At_Stop']
data_dir = '../dublin_data/'
data_file = data_dir + 'siri.20130101.csv'
df = pd.read_csv(data_file, header=None, names=header)

In [20]:
items = os.listdir(data_dir)
items = [item for item in items if (item[:4]=='siri')]
# print (items)

In [21]:
# print df (first 10 rows)
df[:10]

,Timestamp,Line_ID,Direction,Journey_Pattern_ID,Time_Frame,Vehicle_Journey_ID,Operator,Congestion,Lon,Lat,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop
0,1356998403000000,747.0,0,07470001,2012-12-31,3493,SL,0,-6.236852,53.425327,-709,747006,40040,7411.0,0
1,1356998405000000,27.0,0,NaN,2012-12-31,3883,RD,0,-6.233417,53.342232,0,27017,33521,395.0,0
2,1356998407000000,40.0,0,NaN,2012-12-31,2226,HN,0,-6.278250,53.416683,0,40206,33142,6071.0,0
3,1356998407000000,7.0,0,00071003,2012-12-31,6106,D1,0,-6.231633,53.317768,0,7019,43004,3222.0,1
4,1356998411000000,747.0,0,07471001,2012-12-31,3531,SL,0,-6.254617,53.355484,-454,747007,40039,1445.0,0
5,1356998411000000,56.0,0,056A1001,2012-12-31,1830,RD,0,-6.233183,53.342201,0,56001,33488,2379.0,0
6,1356998417000000,25.0,0,025A0001,2012-12-31,2866,CD,0,-6.296867,53.347500,0,25007,33604,4604.0,0
7,1356998423000000,747.0,0,07470001,2012-12-31,3493,SL,0,-6.238668,53.425789,-687,747006,40040,7411.0,0
8,1356998425000000,27.0,0,NaN,2012-12-31,3883,RD,0,-6.233400,53.342232,0,27017,33521,395.0,0
9,1356998427000000,4.0,0,NaN,2012-12-31,4243,HN,0,-6.279000,53.416683,0,4001,43043,7226.0,0


## Main functions

In [22]:
# output a dataframe to specific file
def output(df,outfile):
    df.to_csv(outfile, index=False)

# keep columns by column names
def keep_cols(df,cols):
    drops = header[:]
    [drops.remove(i) for i in cols]
    ndf = df.drop(drops, axis=1)
    return ndf

# keep rows by condition
def keep_rows(df,col,satisfy):
    ndf = df[satisfy(df,col)]
    return ndf

In [23]:
# Return true if col value equal or larger then threshold
def larger(df,col,threshold=1):
    return df[col] >= threshold

## Tests

In [27]:
# test keep_column
cols = ['Timestamp','Line_ID','Lon','Lat','Vehicle_ID']
ndf = keep_cols(df,cols)
nndf = ndf[~ndf.Line_ID.isnull()]
line_ids = set(nndf['Line_ID'].values)

(776631, 5)

In [41]:
ldf = []
line = dict()
for line_id in line_ids:
    sndf = ndf[ndf.Line_ID == line_id]
    vehicle_ids = set(sndf['Vehicle_ID'].values)
    
    # For a specific line_id, find the vehicle_id has largest data point
    for vehicle_id in vehicle_ids:
        tsndf = sndf[sndf.Vehicle_ID == vehicle_id]
        if line_id in line.keys():
            if line[line_id].shape[0] < tsndf.shape[0]:
                line[line_id] = tsndf
        else:
            line[line_id] = tsndf
    # Make lines (from points)
    lsndf = sndf.copy()
    sndf.drop(sndf.index[-1],inplace=True)
    lsndf.drop(lsndf.index[0],inplace=True)
    sndf['toLon'] = lsndf['Lon'].values
    sndf['toLat'] = lsndf['Lat'].values

# Combine all lines to a dataframe
for line_id in line.keys():
    print (line[line_id].shape)
    ldf.append(line[line_id])
odf = pd.concat(ldf)

/Users/haonanl5/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/haonanl5/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/haonanl5/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

(2623, 5)
(2995, 5)
(2601, 5)
(2825, 5)
(2670, 5)
(2342, 5)
(2590, 5)
(2987, 5)
(2502, 5)
(3092, 5)
(2371, 5)
(3036, 5)
(2756, 5)
(2034, 5)
(2036, 5)
(1024, 5)
(2832, 5)
(2913, 5)
(2707, 5)
(2806, 5)
(1962, 5)
(2554, 5)
(1892, 5)
(2838, 5)
(2849, 5)
(2835, 5)
(2961, 5)
(2465, 5)
(2446, 5)
(2096, 5)
(2763, 5)
(2853, 5)
(2737, 5)
(1187, 5)
(2592, 5)
(1280, 5)
(1274, 5)
(1443, 5)
(3014, 5)
(3178, 5)
(2564, 5)
(2208, 5)
(2496, 5)
(2335, 5)
(1566, 5)
(2092, 5)
(780, 5)
(2315, 5)
(2602, 5)
(2313, 5)
(1081, 5)
(1640, 5)
(1052, 5)
(3066, 5)
(1352, 5)
(1748, 5)
(2505, 5)
(2937, 5)


In [72]:
ndf = df[~df.Line_ID.isnull()]
line_ids = list(set(ndf['Line_ID'].values))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.fl

In [79]:
 def line2json(files):
     lines = dict()
     for data_file in files:
         df = pd.read_csv(data_dir + data_file, header=None, names=header)
         ndf = df[~df.Line_ID.isnull()]
         line_ids = [str(i) for i in set(ndf['Line_ID'].values)]
         lines[data_file] = line_ids
         print (lines)
     return lines
files = os.listdir('../dublin_data')
files = [item for item in files if (item[:4]=='siri')]
lines = line2json(files)

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0']}
{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

...

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

{'siri.20130122.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '271.0', '145.0', '17.0', '18.0', '16.0', '272.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '451.0', '68.0', '67.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0', '83.0', '84.0', '220.0', '102.0', '104.0', '747.0', '238.0', '111.0', '114.0', '116.0', '118.0', '120.0', '122.0', '123.0'], 'siri.20130123.csv': ['1.0', '130.0', '4.0', '7.0', '9.0', '11.0', '140.0', '13.0', '14.0', '15.0', '272.0', '145.0', '18.0', '17.0', '16.0', '271.0', '150.0', '151.0', '25.0', '27.0', '29.0', '31.0', '32.0', '33.0', '37.0', '38.0', '39.0', '40.0', '41.0', '42.0', '171.0', '44.0', '46.0', '49.0', '51.0', '53.0', '54.0', '56.0', '59.0', '63.0', '65.0', '66.0', '67.0', '68.0', '451.0', '75.0', '76.0', '332.0', '142.0', '79.0', '331.0'

In [80]:
with open(data_dir + 'route/lines.txt','w') as f:
    f.write(json.dumps(lines, indent=4))
